# Library 및 경로 설정

In [56]:
import os
import json
import re
from pathlib import Path
import shutil
from collections import defaultdict
from tqdm import tqdm

# 프로젝트 루트 경로 설정
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))

# 경로 설정
RAW_DATA_ROOT = os.path.join(current_dir, "raw")
PROCESSED_DATA_ROOT = os.path.join(current_dir, "processed")
STORY_DATA_DIR = os.path.join(PROCESSED_DATA_ROOT, "story_data")

# Direcotry 생성
os.makedirs(STORY_DATA_DIR, exist_ok=True)


In [57]:
print("현재 작업 디렉토리:", os.getcwd())
print("RAW_DATA_ROOT 경로:", RAW_DATA_ROOT)
print("RAW_DATA_ROOT 존재 여부:", os.path.exists(RAW_DATA_ROOT))

# Training 폴더 확인
training_path = os.path.join(RAW_DATA_ROOT, "Training")
print("Training 경로:", training_path)
print("Training 존재 여부:", os.path.exists(training_path))

# Training/01.원천데이터 폴더 확인
src_path = os.path.join(training_path, "01.원천데이터")
print("원천데이터 경로:", src_path)
print("원천데이터 존재 여부:", os.path.exists(src_path))

# 해당 폴더에 실제 파일이 있는지 확인
if os.path.exists(src_path):
    files = os.listdir(src_path)
    print("원천데이터 폴더 내 파일 목록:", files[:10] if len(files) > 10 else files)

현재 작업 디렉토리: /Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data
RAW_DATA_ROOT 경로: /Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw
RAW_DATA_ROOT 존재 여부: True
Training 경로: /Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/Training
Training 존재 여부: True
원천데이터 경로: /Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/Training/01.원천데이터
원천데이터 존재 여부: True
원천데이터 폴더 내 파일 목록: ['TS_03T_사회관계_02S_초등_저학년', 'TS_03T_사회관계_01S_유아', '.DS_Store', 'TS_04T_예술경험_01S_유아', 'TS_02T_자연탐구_01S_유아', 'TS_05T_신체운동_건강_01S_유아', 'TS_02T_자연탐구_02S_초등_저학년', 'TS_04T_예술경험_02S_초등_저학년', 'TS_01T_의사소통_01S_유아', 'TS_01T_의사소통_02S_초등_저학년']


# function 정의

In [58]:
def extract_age_range(path):
    """경로에서 연령대 정보 추출"""
    if "유아" in path:
        return "4-6세"
    else:
        return "7-9세"

def extract_category(path):
    """경로에서 카테고리 정보 추출"""
    categories = {
        "의사소통": "의사소통",
        "자연탐구": "자연탐구",
        "사회관계": "사회관계",
        "예술경험": "예술경험",
        "신체운동_건강": "신체운동_건강"   
    }
    
    for key, value in categories.items():
        if key in path:
            return value
    return "기타"

# JSON file 수집 function

In [59]:
def collect_json_files(data_root):
    """모든 JSON 파일 수집"""
    print("JSON 파일 수집 중....")
    
    json_files = {}
    
    # Training data
    training_src_dir = os.path.join(data_root, "Training", "01.원천데이터")
    training_label_dir = os.path.join(data_root, "Training", "02.라벨링데이터")
    
    # 원천데이터 JSON 파일 수집
    for root, _, files in os.walk(training_src_dir):
        for file in files:
            if file.endswith(".json") and not file.startswith("._"):
                file_path = os.path.join(root, file)
                file_name = os.path.basename(file)
                json_files[file_name] = {"source": file_path, "labeled": None}
    
    # 라벨링데이터 JSON 파일 수집 및 매핑
    for root, _, files in os.walk(training_label_dir):
        for file in files:
            if file.endswith(".json") and not file.startswith("._"):
                file_path = os.path.join(root, file)
                file_name = os.path.basename(file)
                json_files[file_name] = {"source": None, "labeled": file_path}
    
    # Validation data
    validation_src_dir = os.path.join(data_root, "Validation", "01.원천데이터")
    validation_label_dir = os.path.join(data_root, "Validation", "02.라벨링데이터")
    
    # 원천데이터 JSON 파일 수집
    for root, _, files in os.walk(validation_src_dir):
        for file in files:
            if file.endswith('.json') and not file.startswith('._'):
                file_path = os.path.join(root, file)
                file_name = os.path.basename(file)
                json_files[file_name] = {"source": file_path, "labeled": None}
    
    # 라벨링데이터 JSON 파일 수집 및 매핑
    for root, _, files in os.walk(validation_label_dir):
        for file in files:
            if file.endswith('.json') and not file.startswith('._'):
                file_path = os.path.join(root, file)
                file_name = os.path.basename(file)
                if file_name in json_files:
                    json_files[file_name]["labeled"] = file_path
                else:
                    json_files[file_name] = {"source": None, "labeled": file_path}
    
    # SbL 데이터
    sbl_dir = os.path.join(data_root, "SbL")
    
    # SbL JSON 파일 수집
    for root, _, files in os.walk(sbl_dir):
        for file in files:
            if file.endswith('.json') and not file.startswith('._'):
                file_path = os.path.join(root, file)
                file_name = os.path.basename(file)
                if file_name in json_files:
                    json_files[file_name]["sbl"] = file_path
                else:
                    json_files[file_name] = {"source": None, "labeled": None, "sbl": file_path}
    
    print(f"총 {len(json_files)} 개의 JSON 파일을 찾았습니다.")
    return json_files

def read_json_file_with_bom_handling(file_path):
    """BOM이 있는 JSON 파일도 처리할 수 있는 파일 읽기 함수"""
    try:
        # 먼저 utf-8-sig로 시도
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            return json.load(f)
    except UnicodeError:
        # utf-8로 시도
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except json.JSONDecodeError as e:
        # JSON 파싱 오류 처리
        print(f"JSON 파싱 오류: {file_path} - {str(e)}")
        return {}
    except Exception as e:
        # 기타 오류 처리
        print(f"파일 읽기 오류: {file_path} - {str(e)}")
        return {}

## JSON 파일 수집 실행

In [60]:
json_files = collect_json_files(RAW_DATA_ROOT)

# 샘플 파일 확인
list(json_files.items())[:5]


JSON 파일 수집 중....
총 33931 개의 JSON 파일을 찾았습니다.


[('03_03T_02S_9791165430474_67453.json',
  {'source': None,
   'labeled': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/Training/02.라벨링데이터/TL_03T_사회관계_02S_초등_저학년/03_03T_02S_9791165430474_67453.json',
   'sbl': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/SbL/03T_사회관계/02S_초등_저학년/03_03T_02S_9791165430474_67453.json'}),
 ('03_03T_02S_9791155396155_56807.json',
  {'source': None,
   'labeled': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/Training/02.라벨링데이터/TL_03T_사회관계_02S_초등_저학년/03_03T_02S_9791155396155_56807.json',
   'sbl': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/SbL/03T_사회관계/02S_초등_저학년/03_03T_02S_9791155396155_56807.json'}),
 ('03_03T_02S_9791165953003_38237.json',
  {'source': None,
   'labeled': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/Training/02.라벨링데이터/TL_03T_사회관계_02S_초등_저학년/03_03T_02S_979116595300

## 책 단위로 그룹화 함수

In [61]:
def group_by_book(json_files):
    """JSON 파일을 책 단위로 그룹화"""
    print("책 단위로 파일 그룹화 중...")
    books = defaultdict(list)
    
    # ISBN 패턴: 경로 내 9791xxxxxxxxx 패턴 또는 JSON 파일 내용의 isbn 필드
    isbn_pattern = r'979\d{10}'
    
    for file_name, file_paths in json_files.items():
        source_path = file_paths.get("source", file_paths.get("labeled", file_paths.get("sbl", "")))
        if not source_path:
            continue
        
        # 파일 경로에서 ISBN 추출 시도
        isbn_match = re.search(isbn_pattern, source_path)
        if isbn_match:
            isbn = isbn_match.group(0)
            book_id = isbn
            books[book_id].append({
                "file_name": file_name,
                "paths": file_paths
            })
            continue
        
        # 파일 내용에서 ISBN 추출 시도
        try:
            with open(source_path, 'r', encoding='utf-8-sig') as f:
                data = json.load(f)
                if "isbn" in data:
                    isbn = data["isbn"]
                    book_id = isbn
                    books[book_id].append({
                        "file_name": file_name,
                        "paths": file_paths
                    })
                    continue
        except Exception as e:
            print(f"파일 읽기 오류: {source_path} - {str(e)}")
        
        # ISBN을 찾지 못한 경우 파일명을 기반으로 그룹화
        parts = file_name.split('_')
        if len(parts) >= 5:  # 03_01T_01S_9791198465368_54950.json 형식 가정
            book_prefix = '_'.join(parts[:4])
            books[book_prefix].append({
                "file_name": file_name,
                "paths": file_paths
            })
    
    print(f"총 {len(books)} 권의 책이 식별되었습니다.")
    return books

## 책 단위 그룹화 실행

In [62]:
books = group_by_book(json_files)

# 책 단위 그룹화 결과 확인
list(books.items())[:5]





책 단위로 파일 그룹화 중...
총 1563 권의 책이 식별되었습니다.


[('9788954338882',
  [{'file_name': '03_04T_02S_9788954338882_15306.json',
    'paths': {'source': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/Training/01.원천데이터/TS_04T_예술경험_02S_초등_저학년/03_04T_02S_9788954338882_15306.json',
     'labeled': None,
     'sbl': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/SbL/04T_예술경험/02S_초등_저학년/03_04T_02S_9788954338882_15306.json'}},
   {'file_name': '03_04T_02S_9788954338882_15311.json',
    'paths': {'source': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/Training/01.원천데이터/TS_04T_예술경험_02S_초등_저학년/03_04T_02S_9788954338882_15311.json',
     'labeled': None}},
   {'file_name': '03_04T_02S_9788954338882_15307.json',
    'paths': {'source': '/Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/raw/Training/01.원천데이터/TS_04T_예술경험_02S_초등_저학년/03_04T_02S_9788954338882_15307.json',
     'labeled': None}},
   {'file_name': '03_04T_02S_97889

## 책 데이터 처리 함수

In [63]:
def process_book_data(book_id, files, output_dir):
    """책 데이터 처리 및 스토리 단위로 통합"""
    story_data = {
        "story_id": book_id,
        "title": "",
        "tags": "",
        "summary": "",
        "content": "",
        "images": [],
        "metadata": {
            "author": "",
            "publisher": "",
            "isbn": book_id if book_id.startswith('979') else "",
            "published_year": ""
        }
    }
    
    story_text_parts = []
    age_group = ""
    category = ""
    
    # 파일 데이터를 통합하여 스토리 구성
    for file_info in files:
        source_path = file_info["paths"].get("source", "")
        labeled_path = file_info["paths"].get("labeled", "")
        sbl_path = file_info["paths"].get("sbl", "")
        
        # 1. 메타데이터 처리 (원천데이터 또는 라벨링데이터에서)
        metadata_path = source_path if source_path else labeled_path
        if metadata_path:
            try:
                data = read_json_file_with_bom_handling(metadata_path)
                
                # 책 제목 설정 (아직 설정되지 않은 경우)
                if not story_data["title"] and "title" in data:
                    story_data["title"] = data["title"]
                
                # 메타데이터 설정 (아직 설정되지 않은 경우)
                if not story_data["metadata"]["author"] and "author" in data:
                    story_data["metadata"]["author"] = data["author"]
                
                if not story_data["metadata"]["publisher"] and "publisher" in data:
                    story_data["metadata"]["publisher"] = data["publisher"]
                
                if not story_data["metadata"]["isbn"] and "isbn" in data:
                    story_data["metadata"]["isbn"] = data["isbn"]
                
                if not story_data["metadata"]["published_year"] and "publishedYear" in data:
                    story_data["metadata"]["published_year"] = data["publishedYear"]
                
                # 연령대와 카테고리 추출
                if not age_group and metadata_path:
                    age_group = extract_age_range(metadata_path)
                
                if not category and metadata_path:
                    category = extract_category(metadata_path)
                
                # 텍스트 추출 - 더 안전한 방식으로 수정
                image_info = {
                    "image_id": file_info["file_name"].replace('.json', '.jpg'),
                    "caption": "",
                    "text": ""
                }
                
                # None 체크 강화
                if data.get("imageInfo") and isinstance(data["imageInfo"], list) and len(data["imageInfo"]) > 0:
                    image_info_item = data["imageInfo"][0]
                    if isinstance(image_info_item, dict) and "srcText" in image_info_item:
                        text = image_info_item["srcText"]
                        if text:
                            story_text_parts.append(text)
                            image_info["text"] = text
            except Exception as e:
                print(f"메타데이터 처리 오류: {metadata_path} - {str(e)}")
        
        # 2. 라벨링 데이터 처리
        if labeled_path:
            try:
                data = read_json_file_with_bom_handling(labeled_path)
                
                # None 체크 강화
                if data.get("imageInfo") and isinstance(data["imageInfo"], list) and len(data["imageInfo"]) > 0:
                    image_info_item = data["imageInfo"][0]
                    if isinstance(image_info_item, dict) and "imageCaptionInfo" in image_info_item:
                        caption_info = image_info_item["imageCaptionInfo"]
                        if isinstance(caption_info, dict) and "imageCaption" in caption_info:
                            image_info = {
                                "image_id": file_info["file_name"].replace('.json', '.jpg'),
                                "caption": caption_info["imageCaption"],
                                "text": image_info_item.get("srcText", "")
                            }
                            story_data["images"].append(image_info)
            except Exception as e:
                print(f"라벨링 데이터 처리 오류: {labeled_path} - {str(e)}")
        
        # 3. SbL 데이터 처리
        elif sbl_path:
            try:
                data = read_json_file_with_bom_handling(sbl_path)
                
                # imageCaption이 있는지 확인
                if isinstance(data, dict) and "imageCaption" in data:
                    image_info = {
                        "image_id": file_info["file_name"].replace('.json', '.jpg'),
                        "caption": data["imageCaption"],
                        "text": ""
                    }
                    story_data["images"].append(image_info)
            except Exception as e:
                print(f"SbL 데이터 처리 오류: {sbl_path} - {str(e)}")
    
    # 전체 스토리 텍스트 구성
    if story_text_parts:
        story_data["content"] = "\n".join(story_text_parts)
    elif story_data["images"]:
        # 텍스트가 없는 경우 이미지 캡션을 사용
        captions = [img["caption"] for img in story_data["images"] if img["caption"]]
        if captions:
            story_data["content"] = "\n".join(captions)
    
    # 요약 생성
    if story_data["content"]:
        # 간단한 요약: 처음 2-3문장 또는 100자
        sentences = story_data["content"].split('.')
        summary_sentences = sentences[:min(3, len(sentences))]
        story_data["summary"] = '.'.join(summary_sentences)
        if len(story_data["summary"]) > 100:
            story_data["summary"] = story_data["summary"][:100] + "..."
    
    # 태그 생성
    tags = []
    if age_group:
        tags.append(age_group)
    if category:
        tags.append(category)
    if story_data["title"]:
        # 제목에서 키워드 추출 (2글자 이상 단어)
        words = re.findall(r'\w{2,}', story_data["title"])
        tags.extend(words[:2])  # 처음 2개 단어만 태그로 사용
    
    story_data["tags"] = ",".join(tags)
    
    return story_data

## 스토리 데이터 저장 함수

In [64]:
def save_story_data(story_data, output_dir):
    """처리된 스토리 데이터 저장"""
    story_id = story_data["story_id"]
    output_file = os.path.join(output_dir, f"{story_id}.json")
    
    # 문자열로 변환하여 BOM 여부 확인 후 저장
    json_str = json.dumps(story_data, ensure_ascii=False, indent=2)
    if json_str.startswith('\ufeff'):
        json_str = json_str[1:]
        
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(json_str)


# 샘플 처리 Test

In [65]:
sample_book_id = list(books.keys())[0]
sample_files = books[sample_book_id]
sample_story_data = process_book_data(sample_book_id, sample_files, STORY_DATA_DIR)
sample_story_data

{'story_id': '9788954338882',
 'title': '전봉준',
 'tags': '8-9세,예술경험,전봉준',
 'summary': "마을 사람들은 전봉준이 작고 옹골찬 녹두를 닮았다고 '녹두'라고 불렀지요. 농민은 허리가 휘도록 열심히 벼농사를 지었지만 늘 배가 고팠어요. 탐관오리가 세금으로 곡식을 다 거두어 ...",
 'content': '마을 사람들은 전봉준이 작고 옹골찬 녹두를 닮았다고 \'녹두\'라고 불렀지요. 농민은 허리가 휘도록 열심히 벼농사를 지었지만 늘 배가 고팠어요. 탐관오리가 세금으로 곡식을 다 거두어 갔거든요.\n못된 벼슬아치를 몰아내자. 고부 군수 조병갑이 전봉준이 사는 마을을 다스렸어요. 조병갑은 욕심쟁이 탐관오리였답니다.\n소식을 들은 전봉준도 금구에서 사람을 모았지요. "세상을 바로잡을 기회가 왔습니다. 다 같이 보은으로 갑시다!"\n농민들은 전봉준이 써 준 글을 관아에 냈지만 조병갑은 꿈쩍도 하지 않았어요. 화가 난 농민들은 힘을 모아 조병갑을 몰아내기로 했어요. 농민들의 뜻에 따라 전봉준은 계획을 차근차근 세웠어요. 집집마다 사발통문을 돌려 뜻을 같이하는 사람을 모았지요.\n전봉준이 동학의 작은 조직을 책임지고 있을 때였어요. 농민들이 전봉준을 찾아와 말했어요. "조병갑이 멀쩡한 만석보를 다시 쌓으며 돈도 안 주고 우리를 부리더니, 이번엔 물세까지 비싸게 받고 있어요."\n하지만 신식 무기를 쓰는 일본군을 이길 수 없었어요. 더구나 다시 일어난 농민군이 두려워 관군이 일본군과 함께했지요. 결국 농민군은 하나 둘씩 쓰러지기 시작했어요.\n전봉준의 계획도 모른채, 관군은 농민군을 얕잡아 보았어요. 싸움이 시작되자 전봉준이 소리쳤답니다. "다같이 공격!"\n그런데 그때, 언덕에 숨어 있던 농민군이 나타나 학이 날개를 펼치는 것처럼 관군을 에워쌌어요. "장태를 굴려서 총알을 막아라!" 전봉준의 말이 떨어지자마자 농민군은 장태를 굴리며 언덕을 내려왔어요.\n전봉준은 차분하고 당당하게 말했어요. "반역죄는 나라와 백성

# 전체 데이터 처리 실행

In [66]:
processed_count = 0

# 각 책 데이터 처리
for book_id, files in tqdm(list(books.items())[:20], desc="책 처리 중"):  # 처음 20개만 테스트용으로 처리
    # 빈 파일 목록 건너뛰기
    if not files:
        continue
    
    try:
        # 책 데이터 처리
        story_data = process_book_data(book_id, files, STORY_DATA_DIR)
        
        # 유효한 데이터만 저장
        if story_data["title"] and story_data["content"]:
            save_story_data(story_data, STORY_DATA_DIR)
            processed_count += 1
    except Exception as e:
        print(f"책 {book_id} 처리 중 오류 발생: {str(e)}")

print(f"테스트 완료! 총 {processed_count}개의 스토리 데이터가 {STORY_DATA_DIR}에 저장되었습니다.")


책 처리 중: 100%|██████████| 20/20 [00:00<00:00, 466.35it/s]

테스트 완료! 총 20개의 스토리 데이터가 /Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/processed/story_data에 저장되었습니다.


# 전체 데이터 처리 (모든 책 처리)

In [67]:
processed_count = 0

# 각 책 데이터 처리 (전체)
for book_id, files in tqdm(books.items(), desc="책 처리 중"):
    # 빈 파일 목록 건너뛰기
    if not files:
        continue
    
    try:
        # 책 데이터 처리
        story_data = process_book_data(book_id, files, STORY_DATA_DIR)
        
        # 유효한 데이터만 저장
        if story_data["title"] and story_data["content"]:
            save_story_data(story_data, STORY_DATA_DIR)
            processed_count += 1
    except Exception as e:
        print(f"책 {book_id} 처리 중 오류 발생: {str(e)}")

print(f"전처리 완료! 총 {processed_count}개의 스토리 데이터가 {STORY_DATA_DIR}에 저장되었습니다.")


책 처리 중: 100%|██████████| 1563/1563 [00:01<00:00, 1215.33it/s]

전처리 완료! 총 1563개의 스토리 데이터가 /Users/b._.chan/Documents/University/캡스톤디자인/AI/CCB_AI/chatbot/data/processed/story_data에 저장되었습니다.


# 처리된 데이터 샘플 check

In [68]:
processed_files = os.listdir(STORY_DATA_DIR)
print(f"처리된 파일 수: {len(processed_files)}")

sample_file = os.path.join(STORY_DATA_DIR, processed_files[0])
with open(sample_file, 'r', encoding='utf-8-sig') as f:
    sample_data = json.load(f)

sample_data

처리된 파일 수: 1563


{'story_id': '9788956145242',
 'title': '달그락달그락, 보글보글',
 'tags': '4-7세,신체운동_건강,달그락달그락,보글보글',
 'summary': '다음 날 아침, 준수 엄마가 가스레인지를 켜고 냄비에 미역국을 보글보글. 칼과 도마로 호박, 양파, 당근을 탁탁탁, 프라이팬에 지글지글 지그르르. 뒤집개로 잡채를 휘휘, 전을 휙휙',
 'content': '다음 날 아침, 준수 엄마가 가스레인지를 켜고 냄비에 미역국을 보글보글. 칼과 도마로 호박, 양파, 당근을 탁탁탁, 프라이팬에 지글지글 지그르르. 뒤집개로 잡채를 휘휘, 전을 휙휙. 예쁜 접시에 담아내니 준비 끝!',
 'images': [{'image_id': '03_05T_01S_9788956145242_64076.jpg',
   'caption': '준수 엄마가 손에 든 뒤집개로 프라이팬 안의 야채들을 뒤집고 있습니다.',
   'text': '다음 날 아침, 준수 엄마가 가스레인지를 켜고 냄비에 미역국을 보글보글. 칼과 도마로 호박, 양파, 당근을 탁탁탁, 프라이팬에 지글지글 지그르르. 뒤집개로 잡채를 휘휘, 전을 휙휙. 예쁜 접시에 담아내니 준비 끝!'}],
 'metadata': {'author': '김예실',
  'publisher': '(주)한국차일드아카데미',
  'isbn': '9788956145242',
  'published_year': 2015}}